# PowerShell User Group innsalzach 10.06.2021

## Creating your own PowerShell Repository

## What is a PowerShell Repository?

* * *

PowerShell packages are Nuget packages so you need some kind of Nuget Feed to create a PowerShell Repository. 

To access this Nuget Feed you have to register the Nuget Feed locally in PowerShell.

To use a PowerShell Repository there is the Module PowerShell Get that comes preinstalled into Windows.

There is also PowerShellGet V3 which is currently in Preview and will replace the current version of PowerShell Get which is V2.

In a PowerShell repository you can store Scripts, Modules and DSC Resources.   

The most well known Repository for PowerShell resources is the [PowerShell Gallery](https://www.powershellgallery.com/).

## Why should you care about using a PowerShell Repository?
---

If you are in a company where multiple persons / Teams are using PowerShell there has to be some kind of central place where they can store their modules.


Furthermore if you use modules that you are not allowed to use from the PowerShell Gallery (regulatory reasons / no internet connection) you can set up your own internal PowerShell repository. 

You publish your own content and publish external modules you get for example from the PowerShell Gallery. Before you publish the external code to your internal PowerShell Gallery you get the chance to trust the external code by checking / reading the code and then publish it.

## Prepare you System to Use a PowerShell Repository

* * *

In PowerShell Get V2 you need the NuGet Package Provider for using any kind of PowerShell Gallery. The version of PowerShell Get V2 that comes preinstalled with the operating system could be outdated.

For safety reasons update PowerShell Get first.

In [ ]:
# Use TLS 1.2 to download something from the PowerShell Gallery ( Very important!! )
[Net.ServicePointManager]::SecurityProtocol = [Net.ServicePointManager]::SecurityProtocol -bor [Net.SecurityProtocolType]::Tls12

# Install the NuGet Packageprovider
Install-PackageProvider -Name NuGet -Force

# Install the current version of PowerShell Get V2 (Use Force to install the new version side by side of the old one)
Install-Module -Name PowerShellGet -Force

# Restart the shell

In [ ]:
# Import the new PowerShell Get module
Import-Module -Name PowerShellGet

# Get the imported module version (Version 2.2.3 should be there)
Get-Module -Name PowerShellGet

## Create your own internal PowerShell Gallery
---

I show you the easiest way to set up a local PowerShell Repository by using a fileshare

In [ ]:
# Display the current active repositories
Get-PSRepository

In [ ]:
#Create PowerShell repository folder on server
$localPath = 'C:\OnPremRepo' 
New-Item -Path $localPath -ItemType Directory

#Share Powershell repository folder with everyone
$smbShareParam = @{
	Name = 'OnPremRepo'
	Path = $localPath
	Description = 'In House PS Repository'
	FullAccess = 'Everyone'
}
New-SmbShare @smbShareParam

In [ ]:
$remotePath = "\\$env:COMPUTERNAME\OnPremRepo"
$localPsRepoParam = @{
	Name = 'OnPremRepo'
	SourceLocation = $remotePath
	PublishLocation = $remotePath
	InstallationPolicy = 'Trusted'
} 
Register-PSRepository @localPsRepoParam

# Hint: If you have a client / server that is not having the PSGallery you can use
Register-PSRepository -Default

# Check if repository is registered
Get-PSRepository

In [ ]:
#Create a OSInfo module folder 
New-Item C:\OSInfo -ItemType Directory

#Create a very simple module
$moduleCode = @"
Function Get-OsInfo
{
	Get-CimInstance -ClassName win32_OperatingSystem | Select CSName, Caption, Version, OSArchitecture
}
"@ 
$moduleCode | Out-File C:\OSInfo\OSInfo.psm1

#Load and test the OSInfo module
Import-Module -Name C:\OSInfo

#Run module cmdlets
Get-OSInfo

In [11]:
#Create a powershell module manifest for OSInfo Module
$moduleMetaDataParam = @{
	Path = 'C:\OSInfo\OSInfo.psd1'
	RootModule = 'OSInfo.psm1'
	Description = 'CIM operating system information module'
	Author = 'Constantin Hager'
	FunctionsToExport = 'Get-OsInfo'
}
New-ModuleManifest @moduleMetaDataParam

In [12]:
#Check whats on OnPremRepo powershell repository
Find-Module -Repository OnPremRepo

In [13]:
#Do web request over TLS1.2 and Publish Module on local PSRepository (Show in File Explorer)
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
Publish-Module -Path C:\OSInfo -Repository OnPremRepo -Force

In [ ]:
#View the result of OnPremRepo
Find-Module -Repository OnPremRepo

In [ ]:
#Install and test module from local OnPremRepo
Install-Module -Name OSInfo -Repository OnPremRepo -Scope CurrentUser
Get-OsInfo

In [17]:
# If you want to upload a script to your gallery use New-ScriptFileInfo for that. This will create the needed information to find It in the gallery.
# The file should not exist otherwise the cmdlet will fail.
New-ScriptFileInfo -Path "C:\Workfolder\UpoloadScript.ps1" -Description "This is a test upload script"

In [ ]:
# You can test if the script is a valid ScriptFile for your internal gallery
Test-ScriptFileInfo -Path "C:\Workfolder\UpoloadScript.ps1"

In [19]:
# Publish the script to your gallery
Publish-Script -Path "C:\Workfolder\UpoloadScript.ps1" -Repository OnPremRepo

## Alternative PowerShell Galleries
---


- [PowerShell Gallery](https://powerShellgallery.com)

Open Source Gallery
- [NugetGallery](https://github.com/NuGet/NuGetGallery)

SaaS Offerings
- [MyGet](https://www.myget.org/)
- [Azure Artifacts](https://azure.microsoft.com/de-de/services/devops/artifacts/) (You have to authenticate every time you query that repository. This issue will be fixed with PowerShell Get V3)

Deploy as a VM
- [ProGet](https://inedo.com/proget) (a free version is available)


In [20]:
# Housekeeping
Unregister-PSRepository -Name OnPremRepo
Remove-Item -Path C:\OSInfo -Force -Recurse
Remove-Item -Path C:\OnPremRepo -Force -Recurse

## PowerShell Get V3
---

Blog Posts from Sydney Smith (Microsoft PM on PowerShell Team)  

\- [PowerShell Get First Blog Post](https://devblogs.microsoft.com/powershell/powershellget-3-0-preview-1/)

\- [PowerShell Get V3 Preview 10](https://devblogs.microsoft.com/powershell/powershellget-3-0-preview-10-release/)